In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType, PeftModel
import json, torch, gTTS, re, shutil

c:\Users\won39\Desktop\DSBA_project1\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'gTTS'

In [ ]:
# 경로 및 모델명
file_path = "train_data/cleaned_dataset_half.json"
model_name = "beomi/KoAlpaca-Polyglot-5.8B"

# 1. 데이터 로드
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

dataset = Dataset.from_list(data)

# 2. 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # LLaMA 계열에서 필수

# 3. 토크나이징 함수 정의
def tokenize(example):
    prompt = example["input"].strip()
    response = example["output"].strip()
    full_text = prompt + "\n" + response

    tokenized = tokenizer(
        full_text,
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize, remove_columns=["input", "output"])

In [ ]:
# 4. 양자화 설정 + 모델 로드
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=False  # CPU offload 금지
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)


In [ ]:
# 5. LoRA 설정 및 적용
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["attention.query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)


model = get_peft_model(model, lora_config)


In [ ]:
from transformers import AutoTokenizer

base_model = "beomi/KoAlpaca-Polyglot-5.8B"
lora_model_path = "train_data/beomi_llm"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(lora_model_path, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # 필수

# 베이스 모델 로드 + LoRA 적용
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map="auto",
    load_in_8bit=True
)
model = PeftModel.from_pretrained(model, lora_model_path)
model.eval()


In [ ]:
def generate(prompt, max_new_tokens):
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items() if k != "token_type_ids"}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.5,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
def clean_and_trim_response(prompt, generated_text):
    # prompt 이후 부분만 추출
    response = generated_text[len(prompt):].strip()

    # 불필요한 토큰 제거
    response = re.sub(r'(input:|output:)', '', response).strip()

    # 마침표 기준으로 문장 자르기 후 첫 문장만 사용
    sentences = re.split(r'(?<=[.!?])\s+', response)
    return sentences[0] if sentences else response


input_text = "오늘 밥을 뭘 먹는게 좋을까?"
prompt = f"input: {input_text}\noutput:"
generated = generate(prompt, 100)

result = clean_and_trim_response(prompt, generated)
print(result)


In [ ]:
# gTTS 객체 생성
speech = gTTS(text=result, lang='ko', slow=False)
# 음성 파일로 저장
speech.save("tts_output_wav")
